In [2]:
#Required libraries for assignment

!pip install bs4
#!pip install requests

from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd

import folium # map rendering library
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

     |████████████████████████████████| 122kB 15.4MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2021.5.30  |       ha878542_0         136 KB  conda-forge
    certifi-2021.5.30          |   py36h5fab9bb_0         141 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         375 KB

The following NEW packages will be INSTALLED:

  geographic

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")  # create a soup object using the variable 'data'

tables = soup.find_all('table') # in html table is represented by the tag <table>
len(tables)
table_contents=[]


NTable = tables[0]

#Neighborhood_data = pd.DataFrame(columns=["PostalCode", "Borough", "Neighborhood"]) 
#Neighborhood_data

for row in NTable.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# 2nd Part

In [4]:
#Load Longitud and latitude info fom csv file
LL = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")

#Assigning the correct Latitud and longitude to each Postal Code.
LLn = LL.to_numpy() 
dfn = df.to_numpy() 

la = []
lo = []

for i in dfn:
        for n in LLn:
            if n[0] == i[0]:
                la.append(n[1])
                lo.append(n[2])

#Adding Latitud and Longitude to current data frame
                
df['Latitude']  = list(la)
df['Longitude'] = list(lo)
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Part 3

In [5]:
#Average coordinates from Toronto
avg_la = np.mean(la)
avg_lo = np.mean(lo)

# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[avg_la, avg_la], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=False,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#For a reason I do not yet understanbd, I've been spotted somewhere in middle east

In [6]:
#Foursquare crendentials

CLIENT_ID = 'C1WRPSVUXQ5MFGIRI333ZSGWFBLAKFVQPLWJYBKKCBLCXMV5' # your Foursquare ID
CLIENT_SECRET = '4LNXAPZACI4JYBSBY2AAHIWDCGN3LUTOEPWOH3TQ1BZZDYEH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C1WRPSVUXQ5MFGIRI333ZSGWFBLAKFVQPLWJYBKKCBLCXMV5
CLIENT_SECRET:4LNXAPZACI4JYBSBY2AAHIWDCGN3LUTOEPWOH3TQ1BZZDYEH


In [7]:


LIMIT = 70 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

#We will use the average location of the coordinates localiton
url2 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    avg_la, 
    avg_lo, 
    radius, 
    LIMIT)

results = requests.get(url2).json()

In [8]:
#Cathegory and cleaning

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Balsamico,Italian Restaurant,43.701505,-79.397162
1,Noorden Food x Bar,Restaurant,43.702814,-79.397645
2,La Salumeria,Deli / Bodega,43.701255,-79.397117
3,Fit for Life,Salad Place,43.707304,-79.396901
4,Five Doors North,Italian Restaurant,43.702236,-79.397526


In [9]:
#Venues returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

70 venues were returned by Foursquare.


In [10]:
#Unique venues
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 37 uniques categories.


In [11]:
nearby_venues.groupby('categories').count()

,name,lat,lng
categories,,,
Bakery,2,2,2
Bank,3,3,3
Bookstore,2,2,2
Bubble Tea Shop,1,1,1
Burger Joint,1,1,1
Café,3,3,3
Caribbean Restaurant,1,1,1
Coffee Shop,6,6,6
Deli / Bodega,1,1,1


In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [14]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [15]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",2,2,2,2,2,2
Woburn,4,4,4,4,4,4
Woodbine Heights,5,5,5,5,5,5


In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 255 uniques categories.


In [17]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
toronto_grouped.shape

(101, 255)

In [19]:
#Top 3 Common Venues

num_top_venues = 3

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2         Chinese Restaurant  0.25


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.29
1   Coffee Shop  0.14
2  Skating Rink  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0         Bank  0.11
1  Coffee Shop  0.11
2  Pizza Place  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2   Chinese Restaurant  0.25


----Bedford Park, Lawrence Manor East----
                venue  freq
0          Restaurant  0.09
1  Italian Restaurant  0.09
2      Sandwich Place  0.09


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.09
2  Sandwich Place  0.07


----Birch Cliff, Cliffside West----
             venue  freq
0     Skating Rink   0.2
1             Farm   0.2
2  College Stadium   0.2


----Brockton, Parkdale 

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [64]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Chinese Restaurant,Lounge,Breakfast Spot,Cupcake Shop,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
1,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Sandwich Place,Pub,Skating Rink,Coffee Shop,College Cafeteria,Cuban Restaurant,Doner Restaurant,Dog Run
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Ice Cream Shop,Convenience Store,Intersection,Pizza Place,Pharmacy,Health Food Store,Fried Chicken Joint,Sushi Restaurant
3,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Women's Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Italian Restaurant,Restaurant,Sushi Restaurant,Café,Liquor Store,Indian Restaurant,Locksmith,Pub
...,...,...,...,...,...,...,...,...,...,...,...
96,"Willowdale, Newtonbrook",Park,Women's Store,Creperie,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
97,Woburn,Coffee Shop,Korean BBQ Restaurant,Soccer Field,General Entertainment,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
98,Woodbine Heights,Athletics & Sports,Bus Stop,Spa,Beer Store,Skating Rink,College Rec Center,Deli / Bodega,Eastern European Restaurant,Drugstore,College Auditorium
99,York Mills West,Park,Convenience Store,Women's Store,Dim Sum Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner


In [65]:
#Clustering it into 10

kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kint = kmeans.labels_
kint

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 4, 6, 4, 4, 4, 4, 2, 4, 4, 4,
       1, 4, 4, 3, 4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 4,
       4, 4, 5, 4, 3, 4, 4, 4, 3, 4, 4, 3, 0], dtype=int32)

In [66]:
#

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kint)
#neighborhoods_venues_sorted['Cluster Labels'] = list(kint)
toronto_merged = df

#merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged =  toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].fillna(0)
tornto_merged = toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].apply(np.int64)

toronto_merged # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Food & Drink Shop,Park,Fast Food Restaurant,Dessert Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dim Sum Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,4,Coffee Shop,Financial or Legal Service,Hockey Arena,Portuguese Restaurant,Pizza Place,Women's Store,Department Store,Cupcake Shop,Dance Studio,Deli / Bodega
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4,Coffee Shop,Bakery,Café,Park,Pub,Distribution Center,Asian Restaurant,Farmers Market,Spa,Hotel
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4,Clothing Store,Furniture / Home Store,Arts & Crafts Store,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store,Vietnamese Restaurant,Accessories Store,Airport Food Court
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,4,Coffee Shop,Sushi Restaurant,College Cafeteria,Burrito Place,Yoga Studio,Café,Bar,Sandwich Place,Salad Place,Burger Joint
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,3,Park,River,Women's Store,Creperie,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,4,Sushi Restaurant,Japanese Restaurant,Restaurant,Coffee Shop,Gym,Gay Bar,Burrito Place,Fast Food Restaurant,Indian Restaurant,Men's Store
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558,4,Light Rail Station,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Burrito Place,Restaurant,Brewery,Skate Park
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,2,Baseball Field,Women's Store,Electronics Store,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [67]:
map_clusters = folium.Map(location=[avg_la, avg_lo], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
    
#add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyzing Clusters

In [75]:
#Cluster 1 - Red ones
C1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
C1

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,North York,0,Cafeteria,Women's Store,Cuban Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
95,Scarborough,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
#Cluster 2 - Purple ones
C2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
C2

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Food & Drink Shop,Park,Fast Food Restaurant,Dessert Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dim Sum Restaurant
6,Scarborough,1,Fast Food Restaurant,Women's Store,Electronics Store,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
32,Scarborough,1,Playground,Women's Store,Creperie,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
85,Scarborough,1,Playground,Park,Intersection,Women's Store,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant


In [77]:
#Cluster 3 - Dark blue
C3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
C3

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,North York,2,Paper / Office Supplies Store,Baseball Field,Eastern European Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
101,Etobicoke,2,Baseball Field,Women's Store,Electronics Store,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [78]:
#Cluster 4 - Green blue

C4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
C4

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,York,3,Park,Women's Store,Pool,Dessert Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dim Sum Restaurant
35,East York/East Toronto,3,Park,Home Service,Convenience Store,Women's Store,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
52,North York,3,Park,Women's Store,Creperie,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
64,York,3,Park,Jewelry Store,Women's Store,Cuban Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner
66,North York,3,Park,Convenience Store,Women's Store,Dim Sum Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner
91,Downtown Toronto,3,Park,Playground,Trail,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Women's Store
98,Etobicoke,3,Park,River,Women's Store,Creperie,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant


In [79]:
#Cluster 5 - Cyan

C5 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
C5

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,4,Coffee Shop,Financial or Legal Service,Hockey Arena,Portuguese Restaurant,Pizza Place,Women's Store,Department Store,Cupcake Shop,Dance Studio,Deli / Bodega
2,Downtown Toronto,4,Coffee Shop,Bakery,Café,Park,Pub,Distribution Center,Asian Restaurant,Farmers Market,Spa,Hotel
3,North York,4,Clothing Store,Furniture / Home Store,Arts & Crafts Store,Miscellaneous Shop,Coffee Shop,Boutique,Women's Store,Vietnamese Restaurant,Accessories Store,Airport Food Court
4,Queen's Park,4,Coffee Shop,Sushi Restaurant,College Cafeteria,Burrito Place,Yoga Studio,Café,Bar,Sandwich Place,Salad Place,Burger Joint
7,North York,4,Baseball Field,Dessert Shop,Gym,Café,Caribbean Restaurant,Japanese Restaurant,Women's Store,Department Store,Dim Sum Restaurant,Diner
...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,4,Pizza Place,Coffee Shop,Italian Restaurant,Restaurant,Pub,Bakery,Café,Yoga Studio,Park,Snack Place
97,Downtown Toronto,4,Café,Coffee Shop,Sandwich Place,Deli / Bodega,Sushi Restaurant,Japanese Restaurant,Restaurant,Gym,Gastropub,Concert Hall
99,Downtown Toronto,4,Sushi Restaurant,Japanese Restaurant,Restaurant,Coffee Shop,Gym,Gay Bar,Burrito Place,Fast Food Restaurant,Indian Restaurant,Men's Store
100,East Toronto Business,4,Light Rail Station,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Burrito Place,Restaurant,Brewery,Skate Park


In [80]:
#Cluster 6 - Yellow 

C6 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
C6

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Etobicoke,5,Middle Eastern Restaurant,Creperie,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


In [81]:
#Cluster 7 - Orange 

C7 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
C7

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,6,Restaurant,Women's Store,Clothing Store,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant


# Findings

I decided to try 7 clusters taking into account all of the area and surroundings of Toronto instead of just focusing on inner Toronto area. 
 First conclusion is that maybe 7 clusters was too much, probably 4 were just enough to classify

 There is a clear trend that between the area of downtown and Downsview airport a single cluster was created. Founding a bigger variarty of venues on this space. On the other hand, getting further away from this area is likely to find less variety instead. 
 Now I will take a look on the most repeated categories per cluster as 1st common value:

In [84]:
print('Most repead value in cluster 1 (Red ones)   :', C1['1st Most Common Venue'].value_counts().idxmax())
print('Most repead value in cluster 2 (Purple ones): ', C2['1st Most Common Venue'].value_counts().idxmax())
print('Most repead value in cluster 3 (Dark blue)  :', C3['1st Most Common Venue'].value_counts().idxmax())
print('Most repead value in cluster 4 (Green blue) :', C4['1st Most Common Venue'].value_counts().idxmax())
print('Most repead value in cluster 5 (Cyan)       :', C5['1st Most Common Venue'].value_counts().idxmax())
print('Most repead value in cluster 6 (Yellow)     :', C6['1st Most Common Venue'].value_counts().idxmax())
print('Most repead value in cluster 7 (Orange)     :', C7['1st Most Common Venue'].value_counts().idxmax())


Most repead value in cluster 1 (Red ones)   : Cafeteria
Most repead value in cluster 2 (Purple ones):  Playground
Most repead value in cluster 3 (Dark blue)  : Baseball Field
Most repead value in cluster 4 (Green blue) : Park
Most repead value in cluster 5 (Cyan)       : Coffee Shop
Most repead value in cluster 6 (Yellow)     : Middle Eastern Restaurant
Most repead value in cluster 7 (Orange)     : Restaurant


From this we can considered that cluster 1 (Red ones) with the least offer of variety to people.
Cluster 6 doesn't tell me actually to much. But it makes sense due to is far away from downtown area. 

Now good places to make sport are purple and green. 
On the other hand areas on dark blue cluster are good for a baseball match. 
 
Orange cluster is a single line of info, so can not make conclusion about it but the most venue of the area

Good places to go with family and friends are definitely area near to downtown, where cafeteria could be a good place for eating or taking a meal.